In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/maymach09/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/maymach09/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
! pip install "unstructured[all-docs]"

In [1]:
from unstructured.partition.pdf import partition_pdf

In [2]:
raw_pdf_elements=partition_pdf(
    filename="/Users/maymach09/Documents/GenAI09/MacOS/RAG2_Unstructured/data/SoftwareRequirementsSpecification.pdf",                  # mandatory
    strategy="hi_res",                                 # mandatory to use ``hi_res`` strategy
    extract_images_in_pdf=True,                       # mandatory to set as ``True``
    extract_image_block_types=["Image", "Table"],          # optional
    extract_image_block_to_payload=False,                  # optional
    extract_image_block_output_dir="extracted_data",  # optional - only works when ``extract_image_block_to_payload=False``
    )

In [3]:
Header=[]
Footer=[]
Title=[]
NarrativeText=[]
Text=[]
ListItem=[]
for element in raw_pdf_elements:
  if "unstructured.documents.elements.Header" in str(type(element)):
            Header.append(str(element))
  elif "unstructured.documents.elements.Footer" in str(type(element)):
            Footer.append(str(element))
  elif "unstructured.documents.elements.Title" in str(type(element)):
            Title.append(str(element))
  elif "unstructured.documents.elements.NarrativeText" in str(type(element)):
            NarrativeText.append(str(element))
  elif "unstructured.documents.elements.Text" in str(type(element)):
            Text.append(str(element))
  elif "unstructured.documents.elements.ListItem" in str(type(element)):
            ListItem.append(str(element))

In [4]:
img=[]
for element in raw_pdf_elements:
  if "unstructured.documents.elements.Image" in str(type(element)):
            img.append(str(element))

In [5]:
tab=[]
for element in raw_pdf_elements:
  if "unstructured.documents.elements.Table" in str(type(element)):
            tab.append(str(element))

In [6]:
len(img)

7

In [7]:
!pip install langchain_core
!pip install langchain_openai
!pip install langchain
!pip install chromadb

  Using cached langchain_core-0.1.52-py3-none-any.whl.metadata (5.9 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached langsmith-0.1.59-py3-none-any.whl.metadata (13 kB)
  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached tenacity-8.3.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached jsonpointer-2.4-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached orjson-3.10.3-cp311-cp311-macosx_10_15_x86_64.macosx_11_0_arm64.macosx_10_15_universal2.whl.metadata (49 kB)
Using cached langchain_core-0.1.52-py3-none-any.whl (302 kB)
Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
Using cached langsmith-0.1.59-py3-none-any.whl (121 kB)
Using cached packaging-23.2-py3-none-any.whl (53 kB)
Using cached tenacity-8.3.0-py3-none-any.whl (25 kB)
Using cached jsonpointer-2.4-py2.py3-none-any.whl (7.8 kB)
Using cached orjson-3.10.3-cp311-cp311-macosx_10_15_x86_64.macosx_11_0_arm64.macosx_10_15_universal2.whl (253 kB)
  Attempting uni

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [9]:

# Prompt
prompt_text = """You are an assistant tasked with summarizing tables for retrieval. \
    These summaries will be embedded and used to retrieve the raw table elements. \
    Give a concise summary of the table that is well optimized for retrieval. Table:{element} """

In [10]:
prompt = ChatPromptTemplate.from_template(prompt_text)

In [13]:
import os
from dotenv import load_dotenv

In [14]:
load_dotenv()

True

In [15]:
openai_api_key = os.environ.get('OPENAI_API_KEY')

In [16]:

# Text summary chain
model = ChatOpenAI(temperature=0, model="gpt-3.5")
summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

In [17]:
table_summaries = []

In [19]:
table_summaries = summarize_chain.batch(tab, {"max_concurrency": 5})

In [22]:
table_summaries[19]

'The table describes the use case of an Administrator using the Coop Evaluation System to view an Employer\'s submitted evaluation of a student. It details the preconditions, including internet connection and system authentication, and the postconditions, which is the display of the desired evaluation. The table also outlines the normal flow of actions, from selecting "Evaluations" and "Search Submitted/Pending Evaluations", to clicking the "Search" button, selecting the desired evaluation link, and finally selecting the submission for a specific semester.'